# Exploratory Data Analysis (EDA) and Feature Engineering

In this notebook, I will perform exploratory data analysis (EDA) on the cleaned credit card customer data to better understand patterns, relationships, and distributions of variables that may influence customer churn.  

Insights from EDA will guide the creation of meaningful features that could improve the performance of our predictive models.  

### Note on Univariate Analysis

Univariate analysis for both numerical and categorical features was already performed during the **Data Cleaning** stage:

- **Numerical features:**  
  - **Skewness:** Specifically examined `Income`, `CreditLimit`, and `TotalSpend`, as these columns contained missing values and required careful handling for imputation and potential transformation.  
  - **Outliers:** All numerical features were checked for outliers using the **IQR method**. Outliers were **capped** rather than removed to preserve dataset size while reducing the influence of extreme values.

- **Categorical features:**  
  Frequency counts and distributions were assessed.  
  - Low-cardinality features were **one-hot encoded**.  
  - High-cardinality features were **frequency encoded** to retain predictive information without inflating dimensionality.

As a result, the dataset has already been cleaned and transformed at a univariate level, allowing this notebook to focus on:

- Bivariate analysis of features and the target variable  
- Correlation analysis  
- Feature Engineering
- Encoding categorical variables and preparing data for modeling

In [ ]:
# Import necessary libraries
import pandas as pd

# Load the cleaned dataset saved from the previous step
cleaned_data_path = r'..\..\data\processed\credit_card_attrition_cleaned.csv'
df = pd.read_csv(cleaned_data_path)

# Display the first few rows to verify loading
df.head()

In [ ]:
df.columns

In [ ]:
df.shape

## 1. Bivariate analysis of features and the target variable
Goal: Identify features that differ significantly between churned (AttritionFlag=1) and non-churned (AttritionFlag=0) customers.

In [ ]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Count the number of churned and non-churned customers
attrition_counts = df['AttritionFlag'].value_counts()

# Pie chart
plt.figure(figsize=(6,6))
plt.pie(attrition_counts, labels=attrition_counts.index, autopct='%1.1f%%', startangle=90, colors=['#4CAF50', '#F44336'])
plt.title('Distribution of AttritionFlag')
plt.show()

In [ ]:
# Separate features by type
continuous_features = ['Age', 'Income', 'CreditLimit', 'TotalSpend', 'Tenure', 'TotalTransactions']
boolean_features = ['Is_Female', 'MaritalStatus_Divorced', 'MaritalStatus_Married', 
                    'MaritalStatus_Single', 'MaritalStatus_Widowed',
                    'EducationLevel_Bachelor', 'EducationLevel_High School',
                    'EducationLevel_Master', 'EducationLevel_PhD',
                    'CardType_Black', 'CardType_Gold', 'CardType_Platinum', 'CardType_Silver']
engineered_features = [f'Feature_{i}' for i in range(50)] + ['Country_FE']

In [ ]:
# -------------------------------
# 1. Continuous Features
# -------------------------------
import os

# Set folder to save plots
save_folder = r'..\..\reports\figures\EDA_FeatureEng\Continuous_FeaturesVSAttritionFlag'
os.makedirs(save_folder, exist_ok=True)  # Create folder if it doesn't exist

print("Bivariate Analysis: Continuous Features\n")

for col in continuous_features:
    print(col + " vs AttritionFlag")
    print(df.groupby('AttritionFlag')[col].describe(), "\n")
    
    plt.figure(figsize=(6,4))
    sns.boxplot(x='AttritionFlag', y=col, data=df)
    plt.title(f'{col} vs AttritionFlag')
    plt.tight_layout()
    plt.savefig(os.path.join(save_folder, f'{col}_boxplot.png'))
    plt.show() 

*For Continuous features `Age`, `Income`, `CreditLimit`, `TotalSpend`, `Tenure`, `TotalTransactions`,  there are no significant differences in the distributions of continuous features between customers who churned (AttritionFlag = 1) and those who did not (AttritionFlag = 0). This suggests that these continuous features may have limited predictive power for distinguishing churn in this dataset.*

In [ ]:
# -------------------------------
# 2. Boolean Features (Readable)
# -------------------------------
# Set folder to save plots
save_folder = r'..\..\reports\figures\EDA_FeatureEng\Boolean_FeaturesVSAttritionFlag'
os.makedirs(save_folder, exist_ok=True)  # Create folder if it doesn't exist

print("Bivariate Analysis: Boolean Features\n")

for col in boolean_features:
    # Print value counts per AttritionFlag
    print(f'{col} vs AttritionFlag')
    print(pd.crosstab(df[col], df['AttritionFlag'], normalize='columns'), "\n")  # proportions
    
    # Plot grouped bar chart
    plt.figure(figsize=(6,4))
    sns.countplot(x=col, hue='AttritionFlag', data=df, palette='Set2')
    plt.title(f'{col} distribution by AttritionFlag')
    plt.xlabel(f'{col} value')
    plt.ylabel('Count')
    plt.legend(title='AttritionFlag', labels=['No Churn (0)', 'Churn (1)'])
    plt.tight_layout()
    plt.savefig(os.path.join(save_folder, f'{col}_barplot.png'))
    plt.show()

*Same with continuous variables, the boolean features do not show significant differences between Churn and No Churn.*

In [ ]:
# -------------------------------
# Bivariate Analysis: Engineered Features
# -------------------------------
# Set folder to save plots
save_folder = r'..\..\reports\figures\EDA_FeatureEng\Engineered_FeaturesVSAttritionFlag'
os.makedirs(save_folder, exist_ok=True)  # Create folder if it doesn't exist

print("Bivariate Analysis: Engineered Features\n")

for col in engineered_features:
    print(f"{col} vs AttritionFlag")
    # Group statistics
    print(df.groupby('AttritionFlag')[col].mean(), "\n")
    
    # # Boxplot
    plt.figure(figsize=(6,4))
    sns.boxplot(x='AttritionFlag', y=col, data=df)
    plt.title(f'{col} vs AttritionFlag')
    plt.xlabel('AttritionFlag (0=No, 1=Yes)')
    plt.ylabel(col)
    plt.tight_layout()
    
    # Save plot
    plt.savefig(os.path.join(save_folder, f'{col}_boxplot.png'))
    plt.show()

*The bivariate analysis of the engineered features (`Feature_0` to `eature_49` and `Country_FE`) against the target variable AttritionFlag shows that the mean values of each feature are very similar between employees who stayed (`0`) and those who left (`1`).*

### Bivariate Analysis Conclusion
The bivariate analysis shows minimal differences between the features and the target variable (`AttritionFlag`). Continuous, boolean, and engineered features all exhibit similar distributions across the target classes. Overall, no individual feature demonstrates a significant relationship with attrition, indicating that predictive patterns may require modeling feature interactions or more advanced techniques.

## 2. Correlation Analysis

Goal: Discover highly correlated features to inform which features to combine, transform, or drop during feature engineering.

In [ ]:
# Spearman correlation
corr_matrix = df.corr(method='spearman')

pd.set_option('display.max_rows', None) 

# Correlation with target
target_corr = corr_matrix['AttritionFlag'].sort_values(ascending=False)
print(target_corr)

In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(x=target_corr.index, y=target_corr.values)
plt.xticks(rotation=90)
plt.ylabel('Spearman Correlation with AttritionFlag')
plt.title('Feature Correlation with Target')
plt.tight_layout()
plt.show()

*Spearman correlation between all numeric features and `AttritionFlag` shows values extremely close to zero, indicating no significant linear or monotonic relationship. This aligns with the bivariate analysis findings, confirming that none of the features individually differentiate the target.*

In [ ]:
plt.figure(figsize=(20,16))
sns.heatmap(corr_matrix, cmap='coolwarm', center=0, linewidths=0.5)
plt.title('Spearman Correlation Matrix')
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.show()

### Correlation Analysis Conclusion
*There are no notable correlations between feature-feature and feature-target relationships. The few negative correlations observed are expected, as they originate from individual columns that were one-hot encoded.*

## 3. Feature Engineering

Goal: To transform the existing dataset into a richer, more informative set of features that maximizes the model’s ability to predict attrition/churn, despite weak raw correlations.

In [ ]:
df.head()